In [22]:
from tensorflow import keras
import random
import numpy as np

### Chargement du modèle d'autoencoder et séparation de la partie encodeur

In [26]:
autoencoder = keras.models.load_model("models/ae_Min_feature_extr")

Afin d'isoler l'encodeur , on recrée un petit modèle avec une couche d'entrée et les trois couches du modèle d'autoencodeur correspondant à l'encodeur. C'est à dire les trois première couches à l'exception de la couche d'entrée.

In [27]:
inp = keras.Input(shape=(300,))
outp = autoencoder.layers[1](inp)
outp = autoencoder.layers[2](outp)
outp = autoencoder.layers[3](outp)
encoder = keras.Model(inp, outp)

### Encodage du dataset

In [8]:
from gensim.models import KeyedVectors

In [9]:
model = KeyedVectors.load("models/w2vec_model_d300_global_Min")

In [10]:
dim = 300

def encode(msg, model, dim):
    return np.mean([model[word] for word in msg if word in model] or [np.zeros(dim)], axis = 0)

In [11]:
clean_dataset_file = "datasets/global_datasetLem.txt"

In [17]:
with open(clean_dataset_file, "r") as file:
    clean_dataset = file.read().splitlines()

Pour la démonstration on travaille avec un petit échantillon de 50 messages

In [18]:
sample_size = 50

idx_list = random.sample(range(len(clean_dataset)), sample_size)
clean_dataset = [clean_dataset[i] for i in idx_list]

La fonction encode attend une liste de mots on sépare donc les mots des messages.

In [31]:
clean_dataset = [msg.split() for msg in clean_dataset]

In [37]:
dataset_encoded = np.array([encode(msg, model, dim) for msg in clean_dataset])

Une fois l'encodage effectué on a bien 50 vecteurs de 300 composantes dans le dataset encodé.

In [38]:
dataset_encoded.shape

(50, 300)

### Transformation du dataset encodé
On utilise l'encodeur pour réduire le nombre de composantes de chaque vecteur à 16.

In [40]:
dataset_16_features = encoder.predict(dataset_encoded)

On a bien 50 vecteurs a 16 composantes

In [41]:
dataset_16_features.shape

(50, 16)